### Creating database

In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

In [2]:
#creating the database
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              auth_plugin='mysql_native_password',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE HR_Database")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE DATABASE HR_Database


In [3]:
# opening the csv of the dataframe with df dtype and MySQL type 'traduction': 
col_types = pd.read_csv('data/HR_dataset/conversion_sql', index_col=0)

In [4]:
#Copyping the dictionaire of tables creation (table: columns)
col_table = {'working_profile': ['EmpID', 'Term', 'EmploymentStatus', 'DateofHire', 'DateofTermination', 'Salary', 'Department', 'Position', 'ManagerID', 'SpecialProjectsCount'],
            'personal_profile': ['EmpID', 'Employee_Name', 'DOB', 'MaritalStatusID','GenderID', 'FromDiversityJobFairID', 'CitizenDesc', 'RaceDesc', 'HispanicLatino', 'State', 'Zip'],
            'recruitment': ['EmpID', 'RecruitmentSource'],
            'performance_employee': ['EmpID', 'LastPerformanceReview_Date', 'PerformanceScore'],
            'engagement': ['EmpID', 'EngagementSurvey', 'EmpSatisfaction'],
            'absenteeism': ['EmpID', 'DaysLateLast30', 'Absences'],
            'attrition': ['EmpID', 'EmploymentStatus', 'DateofTermination', 'TermReason'],
            'manager': ['ManagerID', 'ManagerName'],
            'marital_status': ['MaritalStatusID', 'MaritalDesc', 'Married'],
            'gender': ['GenderID', 'Sex']}

In [6]:
#automating the tables creation:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='HR_Database')

mycursor = cnx.cursor()
try:
    for table_name, columns in col_table.items():
        columns_sql = ', '.join([f'{column} {col_types.loc[col_types["index"] == column, "SQL_conv"].values[0]}' for column in columns])
        mycursor.execute(f'CREATE TABLE {table_name} ({columns_sql}, PRIMARY KEY ({columns[0]}));')
        print(mycursor)
        
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE TABLE working_profile (EmpID INT,..
CMySQLCursor: CREATE TABLE personal_profile (EmpID INT..
CMySQLCursor: CREATE TABLE recruitment (EmpID INT, Rec..
CMySQLCursor: CREATE TABLE performance_employee (EmpID..
CMySQLCursor: CREATE TABLE engagement (EmpID INT, Enga..
CMySQLCursor: CREATE TABLE absenteeism (EmpID INT, Day..
CMySQLCursor: CREATE TABLE attrition (EmpID INT, Emplo..
CMySQLCursor: CREATE TABLE manager (ManagerID INT, Man..
CMySQLCursor: CREATE TABLE marital_status (MaritalStat..
CMySQLCursor: CREATE TABLE gender (GenderID INT, Sex V..


In [10]:
#importamos cada tabla para insertar los datos:
"""for k in col_table.keys():
    with open(f'data/HR_dataset/{k}.csv','r') as file:
        globals()[k] = file.readlines()"""

In [ ]:
#inserting data:

"""cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='HR_Database')

        mycursor = cnx.cursor()

        for k, v in col_table.items():
            
            sql = """INSERT INTO txt (index_txt, q3, q4, q5, q6, q7, q8, q9, q11, q12, q13, q14, q15, q16, q17, 
                                q20, q21, q22, q23, q24, q25, q26, q32, q33, q34, q35, q41)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
            val = self.creacion_tabla_txt(file_source)[1:]

        try: 
            mycursor.executemany(sql, val)
            cnx.commit()
            print(mycursor.rowcount, "registro/s insertado/s.")

        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)"""

In [7]:
#creating foreign keys:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='HR_Database')

mycursor = cnx.cursor()
try:

    mycursor.execute("""ALTER TABLE `working_profile` 
                     ADD CONSTRAINT `fk_working_personal` FOREIGN KEY (`EmpID`) REFERENCES `personal_profile` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE, 
                     ADD CONSTRAINT `fk_working_recruitment` FOREIGN KEY (`EmpID`) REFERENCES `recruitment` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE, 
                     ADD CONSTRAINT `fk_working_performance_employee` FOREIGN KEY (`EmpID`) REFERENCES `performance_employee` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE, 
                     ADD CONSTRAINT `fk_working_engagement` FOREIGN KEY (`EmpID`) REFERENCES `engagement` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE, 
                     ADD CONSTRAINT `fk_working_absenteeism` FOREIGN KEY (`EmpID`) REFERENCES `absenteeism` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE, 
                     ADD CONSTRAINT `fk_working_attrition` FOREIGN KEY (`EmpID`) REFERENCES `attrition` (`EmpID`) ON DELETE RESTRICT ON UPDATE CASCADE; 
                     ALTER TABLE `manager` ADD CONSTRAINT `fk_manager_working` FOREIGN KEY (`ManagerID`) REFERENCES `working_profile` (`ManagerID`) ON DELETE RESTRICT ON UPDATE CASCADE; 
                     ALTER TABLE `marital_status` ADD CONSTRAINT `fk_marital_personal` FOREIGN KEY (`MaritalStatusID`) REFERENCES `personal_profile`(`MaritalStatusID`) ON DELETE RESTRICT ON UPDATE CASCADE; 
                     ALTER TABLE `gender` ADD CONSTRAINT `fk_gender_personal` FOREIGN KEY (`GenderID`) REFERENCES `personal_profile` (`GenderID`) ON DELETE RESTRICT ON UPDATE CASCADE;""")
    print(mycursor)
    
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: ALTER TABLE `working_profile` 
         ..
